# Andreas sitt workplace, GTFO!!

Jacobi 22

In [171]:

# main del #################################################################################
#  får inn verdier for y og bus  

# Import necessary classes from the 'classes' module
from classes import *
import pandas as pd
import os

# Get the absolute path to the current directory
pathToFile = os.path.abspath(os.curdir)

# Read network configuration data from an Excel file
df_line_data = pd.read_excel(f"{pathToFile}\\files\\network_configuration.xlsx", sheet_name="Line_data")
df_bus_data = pd.read_excel(f"{pathToFile}\\files\\network_configuration.xlsx", sheet_name="Bus_data")

# Display the line data
#display(df_line_data)

# Convert data frames to dictionaries
line_data = df_line_data.to_dict(orient='records')
bus_data = df_bus_data.to_dict(orient='records')

# Display the bus data
display(df_bus_data)

# Initialize a list to store line admittances
line_admittences = []

# Loop through the line data and build Line objects
for line_dict in line_data:
    line = Line(
        bus_p=line_dict['From line'],
        bus_q=line_dict['To line'],
        impedance=complex(line_dict['R[pu]'], line_dict['X[pu]']),
        half_line_charging_admittance=complex(line_dict['Half Line Charging Admittance'])
    )
    line_admittences.append(line.build_Line_Y_bus())

#Display the line admittances
#display(line_admittences)

# Create a Bus object for bus 1 with specific parameters
bus1 = Bus(bus_id=1, voltage_magnitude=20, voltage_angle=0)

# Uncomment the following line if you want to print the bus1 object
# print(bus1)

# slutt main del ##################################################################

a_bus_data = df_bus_data.values

#display(a_bus_data)

df_overview = pd.DataFrame(columns=['Bus number', 'Type', 'Known_1', 'Known_2', 'Unknown_1', 'Unknown_2'], index=[df_bus_data.index])

print(df_bus_data.loc[0, 'Generation (MW)'])

for i in df_bus_data.index:
    df_overview.loc[i, 'Bus number'] = df_bus_data.loc[i, 'Bus Code']

    try:
        generation_mw = (df_bus_data.loc[i, 'Generation (MW)'])
        load_mw = (df_bus_data.loc[i, 'Load (MW)'])
        generation_mvar = (df_bus_data.loc[i, 'Generation (MVAr)'])
        load_mvar = (df_bus_data.loc[i, 'Load (MVAr)'])
        bus_voltage = (df_bus_data.loc[i, 'Assumed bus voltage (pu)'])
    except ValueError:
        # Handle non-integer values
        generation_mw = "-"
        load_mw = "-"
        generation_mvar = "-"
        load_mvar = "-"
        bus_voltage = "-"


    if generation_mw != "-" and load_mw != "-" and generation_mvar != "-" and load_mvar != "-":
        df_overview.loc[i, 'Type'] = 'P' + 'Q'
        df_overview.loc[i, 'Known_1'] = 'P'
        df_overview.loc[i, 'Known_2'] = 'Q'
        df_overview.loc[i, 'Unknown_1'] = 'V'
        df_overview.loc[i, 'Unknown_2'] = 'θ'
    
    elif bus_voltage != "-" and (generation_mw != "-" and load_mw != "-"):
        df_overview.loc[i, 'Type'] = 'P' + 'V'
        df_overview.loc[i, 'Known_1'] = 'P'
        df_overview.loc[i, 'Known_2'] = 'V'
        df_overview.loc[i, 'Unknown_1'] = 'Q' 
        df_overview.loc[i, 'Unknown_2'] = 'θ'

    else:
        df_overview.loc[i, 'Type'] = "Slack"
        df_overview.loc[i, 'Known_1'] = 'V'
        df_overview.loc[i, 'Known_2'] = 'θ'
        df_overview.loc[i, 'Unknown_1'] = 'P'
        df_overview.loc[i, 'Unknown_2'] = 'Q'






data_dict_P = {}

for i in df_bus_data.index:

    while(df_bus_data.loc[i, 'Load (MW)']) == '-' or (df_bus_data.loc[i, 'Generation (MW)']) == '-' :
        i+=1

    # Create dynamic variable names
    new_name_P = f"P_{i+1}"
    # Example value for the variable
    valueP = float(df_bus_data.loc[i, 'Load (MW)']) - float(df_bus_data.loc[i, 'Generation (MW)'])
    # Store the value in the dictionary with the dynamically created name
    data_dict_P[new_name_P] = valueP

data_dict_Q = {}

for i in df_bus_data.index:

    while (df_bus_data.loc[i, 'Load (MVAr)']) == '-' or (df_bus_data.loc[i, 'Generation (MVAr)']) == '-':
        i+=1
        
    # Create dynamic variable names
    new_name_Q = f"Q_{i+1}"
    # Example value for the variable
    valueQ = float(df_bus_data.loc[i, 'Load (MVAr)']) - float(df_bus_data.loc[i, 'Generation (MVAr)'])
    # Store the value in the dictionary with the dynamically created name
    data_dict_Q[new_name_Q] = valueQ 







# Display the updated df_overview
display(df_overview)


display(data_dict_P)
display(data_dict_Q)

df_known_P  = pd.DataFrame.from_dict(data_dict_P, orient='index')
display(df_known_P)

df_known_Q  = pd.DataFrame.from_dict(data_dict_Q, orient='index',)
display(df_known_Q)

df_known = pd.concat([df_known_P, df_known_Q], axis = 0)
display(df_known)

df_known_pu = df_known / 100
display(df_known_pu)


,Bus Code,Assumed bus voltage (pu),Angle,Generation (MW),Generation (MVAr),Load (MW),Load (MVAr)
0,1,1.06,0,-,-,0,0
1,2,1.00,0,40,-,20,10
2,3,1.00,0,0,0,45,15
3,4,1.00,0,0,0,40,5
4,5,1.00,0,0,0,60,10


-


,Bus number,Type,Known_1,Known_2,Unknown_1,Unknown_2
0,1,Slack,V,θ,P,Q
1,2,PV,P,V,Q,θ
2,3,PQ,P,Q,V,θ
3,4,PQ,P,Q,V,θ
4,5,PQ,P,Q,V,θ


{'P_2': -20.0, 'P_3': 45.0, 'P_4': 40.0, 'P_5': 60.0}

{'Q_3': 15.0, 'Q_4': 5.0, 'Q_5': 10.0}

,0
P_2,-20.0
P_3,45.0
P_4,40.0
P_5,60.0


,0
Q_3,15.0
Q_4,5.0
Q_5,10.0


,0
P_2,-20.0
P_3,45.0
P_4,40.0
P_5,60.0
Q_3,15.0
Q_4,5.0
Q_5,10.0


,0
P_2,-0.20
P_3,0.45
P_4,0.40
P_5,0.60
Q_3,0.15
Q_4,0.05
Q_5,0.10
